In [1]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from transformers import BertTokenizer,TFBertModel,TFBertForSequenceClassification

In [2]:
friend_data_path = r'/home/ning/dataset'
bert_path = r"/home/ning/bert_conf"
cache_dir=r'/home/ning/bert_conf/bert-base-uncased-cache'



train_df = pd.read_pickle(os.path.join(friend_data_path,'friends_train.pkl'))
test_df = pd.read_pickle(os.path.join(friend_data_path,'friends_test.pkl'))

train_df = train_df[train_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]
test_df = test_df[test_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]

tokenizer = BertTokenizer.from_pretrained(os.path.join(bert_path,'vocab_friends.txt'))
model = TFBertForSequenceClassification.from_pretrained(os.path.join(bert_path,'tf_model.h5'),config = os.path.join(bert_path,'config.json'),num_labels=4)


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Some weights of the model checkpoint at /home/ning/bert_conf/tf_model.h5 were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at /home/ning/bert_conf/tf_model.h5 and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task 

In [3]:
def encode_sentence_with_speaker(speaker,utterance,tokenizer,sos):
    if sos:
        tokens = ['[CLS]']
    else:
        tokens = []

    if speaker not in ['other','None']:
        spk_token = '['+speaker+']'
        tokens.append(spk_token)
        tokens.append('[says]')
        tokens.extend(list(tokenizer.tokenize(utterance)))
        tokens.append('[SEP]')
    
    else:
        tokens.extend(list(tokenizer.tokenize(utterance)))
    
    return tokenizer.convert_tokens_to_ids(tokens)
    
    


def bert_encode(dataframe, tokenizer):
    num_examples = len(dataframe.index)
    sentence1 = tf.ragged.constant([encode_sentence_with_speaker(s[0],s[1],tokenizer,True) for s in dataframe.values])

    sentence2 = tf.ragged.constant([encode_sentence_with_speaker(s[2],s[3],tokenizer,False) for s in dataframe.values])
    
    input_word_ids = tf.concat([sentence1, sentence2], axis=-1)
    
    input_mask = tf.ones_like(input_word_ids).to_tensor()
    
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat([type_s1, type_s2], axis=-1).to_tensor()
    
    
    inputs = {
        'input_ids': input_word_ids.to_tensor(),
        'attention_mask': input_mask,
        'token_type_ids': input_type_ids}

    return inputs

In [4]:
train_target = train_df['emotion']
test_target = test_df['emotion']
train_features = bert_encode(train_df, tokenizer)
test_features = bert_encode(test_df, tokenizer)

ems = train_target.unique()
def convert(emotion):
    return np.where(ems == emotion)[0][0]

train_labels = np.array(list(map(convert,train_target)))
test_labels = np.array(list(map(convert,test_target)))

In [5]:
batch_size = 16
num_labels = 4
epochs = 3
weight = np.array([[sum(train_target==e) for e in ems]]*batch_size)
weight = weight[0].sum()/weight
sample_weight = tf.constant(weight,dtype=tf.float32)

def NLL(y_true,y_pred,sample_weight = sample_weight):
    
    node1 = tf.nn.softmax(y_pred)
    node2 = sample_weight*node1
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    return loss(y_true,node2)



from sklearn.metrics import f1_score
optimizer = tf.keras.optimizers.Adam(learning_rate=2.5e-6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]



In [50]:

model_save_path = r'/home/ning/models_ckpt/'
bert_callback = tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,
                                                  save_weights_only=True,
                                                  save_best_only=True)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

history = model.fit(train_features,train_labels,
                    batch_size = batch_size,
                    epochs=1,
                    validation_data=(test_features,test_labels),
                    callbacks=[bert_callback]
                    )

446/446 [==============================] - 77s 174ms/step - loss: 0.3840 - accuracy: 0.8647 - val_loss: 0.5055 - val_accuracy: 0.8281


In [11]:
train_df.head()

,speaker,utterance,previous_speaker,causal_utterance,emotion
2,Ross,"Or! Or, we could go to the bank, close our acc...",Monica,What?,neutral
3,Chandler,You're a genius!,Ross,"Or! Or, we could go to the bank, close our acc...",joy
4,Joey,"Aww, man, now we won't be bank buddies!",Chandler,You're a genius!,sadness
5,Chandler,"Now, there's two reasons.",Joey,"Aww, man, now we won't be bank buddies!",neutral
6,Phoebe,Hey.,Chandler,"Now, there's two reasons.",neutral


In [40]:
s = ''
for i in tokenizer.convert_ids_to_tokens(train_features['input_ids'][0].numpy()):
    if i != '[PAD]':
        s+=i
        s+=' '
print(s)
                                    

[CLS] [Ross] [says] or ! or , we could go to the bank , close our accounts and cut them off at the source . [SEP] [Monica] [says] what ? [SEP] 


In [6]:

loss_object= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=2.5e-6)

batch_size = 16
num_labels = 4
TPs = np.zeros(num_labels)
FPs = np.zeros(num_labels)
FNs = np.zeros(num_labels) 

def cal_score(results, label_seq):
    for i in range(len(label_seq)):
        a = results[i]
        b = label_seq[i]
        if b == a:
            TPs[a] += 1
        else:
            FPs[a] += 1
            FNs[b] += 1
    precisions = TPs/(TPs+FPs) if TPs+FPs!=0 else 0
    recalls = TPs/(TPs+FNs) if TPs+FNs!=0 else 0
    
    F1s = 2*precisions*recalls/(precisions+recalls)
    micro_p = np.sum(TPs)/np.sum(TPs+FPs) if TPs+FPs!=0 else 0
    micro_r = np.sum(TPs)/np.sum(TPs+FNs) if TPs+FNs!=0 else 0
    micro_f1 = 2*micro_p*micro_r/(micro_p+micro_r) if micro_p+micro_r!=0 else 0
    return precisions,recalls,F1s,micro_p,micro_r ,micro_f1

def show_perfomence(precisions,recalls,F1s,micro_p,micro_r ,micro_f1):
    print("precisions:{}".format(precisions))
    print("recalls:{}".format(recalls))
    print("F1s:{}".format(F1s))
    print("micro_f1:{}".format(micro_f1))
    
    return

def calc_confusion_matrix(matrix,predict,true):
    
    for i in range(len(predict)):
        matrix[true[i]][predict[i]] += 1
    return matrix


def train(train_f,train_l,test_f,test_l):

    
    confusion_matrix = np.zeros((4,4),dtype=np.float32)
    
    batch_size = 16
    num_labels = 4
    epochs = 3
    weight = np.array([[sum(train_target==e) for e in ems]]*batch_size)
    weight = weight[0].sum()/weight
    sample_weight = tf.constant(weight,dtype=tf.float32)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=2.5e-6)
    
    for i in range(epochs):
        print("Training for epoch {}".format(i))
        for step in range(len(train_l)//batch_size):
            train_f_batch = {'input_ids': train_f['input_ids'][batch_size*step:batch_size*(step+1)],
                    'attention_mask': train_f['attention_mask'][batch_size*step:batch_size*(step+1)],
                    'token_type_ids': train_f['token_type_ids'][batch_size*step:batch_size*(step+1)]}
            train_l_batch = train_l[batch_size*step:batch_size*(step+1)]

            with tf.GradientTape() as tape:
                prediction = model(train_f_batch,training = True)
                pred_label = np.argmax(prediction[0].numpy(),axis=1)
                
                loss_value  = loss_object(y_true=train_l_batch, y_pred=prediction)
                
                
            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        
            if not step%50:
                print("Epoch {:03d}: Step:{:03d}//{:03d} Loss: {:.3f} ".format(i,step,len(train_l)//batch_size,loss_value))
                print(confusion_matrix)
                
            test_f_batch = {'input_ids': test_f['input_ids'][batch_size*step:batch_size*(step+1)],
                    'attention_mask': test_f['attention_mask'][batch_size*step:batch_size*(step+1)],
                    'token_type_ids': test_f['token_type_ids'][batch_size*step:batch_size*(step+1)]}
            test_l_batch = test_l[batch_size*step:batch_size*(step+1)]
            prediction = model(test_f_batch,training = False)
            pred_label = np.argmax(prediction[0].numpy(),axis=1)
            
            if len(test_l_batch)==len(pred_label):
                confusion_matrix = calc_confusion_matrix(confusion_matrix,pred_label,test_l_batch)
                            
    return confusion_matrix



In [12]:
confusion_matrix = train(train_features,train_labels,test_features,test_labels)

confusion_matrix

Training for epoch 0
Epoch 000: Step:000//445 Loss: 0.994 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Epoch 000: Step:050//445 Loss: 0.811 
[[494.   2.   0.   0.]
 [190.   3.   0.   0.]
 [ 31.   2.   0.   0.]
 [ 78.   0.   0.   0.]]
Epoch 000: Step:100//445 Loss: 0.633 
[[1046.    2.    0.    0.]
 [ 352.    3.    0.    0.]
 [  65.    2.    0.    0.]
 [ 130.    0.    0.    0.]]
Epoch 000: Step:150//445 Loss: 1.073 
[[1582.    4.    0.    0.]
 [ 477.    3.    0.    0.]
 [ 117.    2.    0.    0.]
 [ 183.    0.    0.    0.]]
Epoch 000: Step:200//445 Loss: 1.043 
[[1582.    4.    0.    0.]
 [ 477.    3.    0.    0.]
 [ 117.    2.    0.    0.]
 [ 183.    0.    0.    0.]]
Epoch 000: Step:250//445 Loss: 1.248 
[[1582.    4.    0.    0.]
 [ 477.    3.    0.    0.]
 [ 117.    2.    0.    0.]
 [ 183.    0.    0.    0.]]
Epoch 000: Step:300//445 Loss: 0.556 
[[1582.    4.    0.    0.]
 [ 477.    3.    0.    0.]
 [ 117.    2.    0.    0.]
 [ 183.    0.    0.    0.]]
Epoch 000: Ste

KeyboardInterrupt: 

In [1]:
sum(train_target=='neutral')#  69%  4936

NameError: name 'train_target' is not defined

In [97]:
sum(train_target=='sadness')

379

In [98]:
sum(train_target=='anger')

576

In [99]:
sum(train_target=='joy')

1230

In [102]:
ems

array(['neutral', 'joy', 'sadness', 'anger'], dtype=object)

In [5]:
import numpy as np
weights = np.array([4936,1230,379,576])

In [7]:
weights = weights.sum()/weights

In [8]:
weights


array([ 1.44266613,  5.78943089, 18.78891821, 12.36284722])

In [109]:
weight = np.array([[sum(train_target==e) for e in ems]]*batch_size)


In [ ]:
weight[0].sum()/weight
weight = weight/weight[0].sum

In [10]:
model({'input_ids': train_features['input_ids'][0:5],
        'attention_mask': train_features['attention_mask'][0:5],
        'token_type_ids': train_features['token_type_ids'][0:5]})[0].numpy()

array([[ 0.689963  , -0.12482441, -0.53783125, -0.4852844 ],
       [ 0.7734618 , -0.17940116, -0.55217063, -0.46465898],
       [ 0.6775636 , -0.10690157, -0.69079584, -0.63610595],
       [ 0.7091562 , -0.04609575, -0.18738848, -0.40678862],
       [ 0.7497835 , -0.12705968, -0.6054592 , -0.5364008 ]],
      dtype=float32)

In [7]:
test_features['input_ids'].shape

TensorShape([7123, 115])

# 保存模型，重新加载

In [56]:
model.save('/home/ning/models.ckpt')

INFO:tensorflow:Assets written to: /home/ning/models.ckpt/assets


In [55]:
model.save_pretrained('/home/ning/models.ckpt/')

model.save_weights('/home/ning/models.ckpt/')


In [57]:
new_model = tf.keras.models.load_model('/home/ning/models.ckpt')

In [68]:
model.layers[0].set_weights(new_model.layers[0].get_weights())

In [69]:
model.predict({'input_ids': train_features['input_ids'][0:5],
        'attention_mask': train_features['attention_mask'][0:5],
        'token_type_ids': train_features['token_type_ids'][0:5]})

(array([[ 2.7381787 , -1.7181199 , -1.5035897 , -0.49402303],
        [-0.44213945,  2.3243062 , -2.1584902 ,  0.15950564],
        [-0.6702639 , -0.8094952 , -0.41541195,  1.723997  ],
        [ 2.8739057 , -2.1094537 , -0.8364453 , -1.2881769 ],
        [ 3.4454439 , -1.5704695 , -1.3068649 , -1.8356565 ]],
       dtype=float32),)

In [31]:
m = TFBertModel.from_pretrained(os.path.join(bert_path,'tf_model.h5'),config = os.path.join(bert_path,'config.json'))


Some weights of the model checkpoint at /home/ning/bert_conf/tf_model.h5 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at /home/ning/bert_conf/tf_model.h5.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [34]:
m.layers[0].set_weights(model.layers[0].get_weights())

In [22]:





model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)


m.fit(train_features,train_labels,batch_size = batch_size,epochs=epochs,validation_data=(test_features,test_labels))

Epoch 1/3


ValueError: in user code:

    /home/ning/envs/anaconda3/envs/tfcpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/ning/envs/anaconda3/envs/tfcpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ning/envs/anaconda3/envs/tfcpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ning/envs/anaconda3/envs/tfcpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ning/envs/anaconda3/envs/tfcpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /home/ning/envs/anacond

# 自己建模型

In [1]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import tensorflow as tf
from transformers import BertTokenizer,TFBertModel,TFBertForSequenceClassification,BertConfig,TFBertForPreTraining

In [5]:
friend_data_path = r'/home/ning/dataset'
bert_path = r"/home/ning/bert_conf"
cache_dir=r'/home/ning/bert_conf/bert-base-uncased-cache'

ckpt_path = r'/home/ning/models.ckpt'

train_df = pd.read_pickle(os.path.join(friend_data_path,'friends_train.pkl'))
test_df = pd.read_pickle(os.path.join(friend_data_path,'friends_test.pkl'))

train_df = train_df[train_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]
test_df = test_df[test_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]

train_df_emo = train_df[train_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]
test_df_emo = test_df[test_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]

tokenizer = BertTokenizer.from_pretrained(os.path.join(bert_path,'vocab_friends.txt'))

# model_trained = TFBertForSequenceClassification.from_pretrained(os.path.join(ckpt_path,'friendsbert_no_pretrained.h5'),config = os.path.join(ckpt_path,'config.json'),num_labels=4)
model = TFBertModel.from_pretrained(os.path.join(ckpt_path,'friendsbert_pretrained.h5'),config = os.path.join(ckpt_path,'config.json'),num_labels=4)
# model = TFBertForSequenceClassification.from_pretrained(os.path.join(bert_path,'tf_model.h5'),config = os.path.join(bert_path,'config.json'),num_labels=4)


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Some weights of the model checkpoint at /home/ning/models.ckpt/friendsbert_pretrained.h5 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at /home/ning/models.ckpt/friendsbert_pretrained.h5.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
b = FriendsBert()

Some weights of the model checkpoint at /home/ning/models.ckpt/friendsbert_pretrained.h5 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at /home/ning/models.ckpt/friendsbert_pretrained.h5.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [8]:
b.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.